<a href="https://colab.research.google.com/github/OMNI-ML/pytorch-grad-cam-anim/blob/adapt-basecam-to-support-cam_anim/_CAManim_animating_end2end_activation_maps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Animating End-to-End Network Actication Maps

Authors: Emily Kaczmarek, Olivier Miguel, Kevin Dick

---

#### Resources:
* GitHub repo to push our changes to: https://github.com/jacobgil/pytorch-grad-cam
* Command for ffmpeg: `ffmpeg -framerate 5 -pattern_type glob -i "*.png" -c:v libx264 -r 30 -vf "scale=1200:-2,format=yuv420p" -movflags +faststart output.mp4`

TODO: write fancy things here

#### Hackathon Day 1:
* Determine a few example benchmark image datasets that we demo
  * MNIST
  * whatever GradCAM paper used
  * Anotherrrrr
* Determine what pre-trained model architecture we use:
 * DenseNet (no isses in layer name



# Olivier


## Package setup

In [1]:
!git clone https://github.com/OMNI-ML/pytorch-grad-cam-anim.git
!ls

Cloning into 'pytorch-grad-cam-anim'...
remote: Enumerating objects: 4641, done.
remote: Counting objects: 100% (3615/3615), done.
remote: Compressing objects: 100% (354/354), done.
remote: Total 4641 (delta 3017), reused 3562 (delta 2968), pack-reused 1026
Receiving objects: 100% (4641/4641), 1.07 GiB | 38.72 MiB/s, done.
Resolving deltas: 100% (3583/3583), done.
pytorch-grad-cam-anim  sample_data


In [2]:

# the % command makes the cd last beyond this cell / line
%cd pytorch-grad-cam-anim
!ls
!git checkout adapt-basecam-to-support-cam_anim


# !python setup.py install - avoid this
!pip install .
!pip install -r requirements_CAManim.txt
!pip install ffmpeg-python

/content/pytorch-grad-cam-anim
cam.py	  pyproject.toml    requirements.txt  tests
examples  pytorch_grad_cam  setup.cfg	      tutorials
LICENSE   README.md	    setup.py	      usage_examples
Branch 'adapt-basecam-to-support-cam_anim' set up to track remote branch 'adapt-basecam-to-support-cam_anim' from 'origin'.
Switched to a new branch 'adapt-basecam-to-support-cam_anim'
Processing /content/pytorch-grad-cam-anim
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for grad-cam: filename=grad_cam-1.4.6-py3-none-any.whl size=44510 sha256=a7f25b775957cfda6fbce566ea531b2e82e0dd5078747a40803f2b6c1803c272
  Stored in directory: /root/.cache/pip/wheels/5e/e9/6a/e6fc4e1d28c7938e9ff20505457d082d0fb87579dfe1f61668
Successfully built grad-cam
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 9.7 MB/s eta 0:00:00


In [3]:
!git log --oneline --max-count=10 # --reverse

2dec3b2 (HEAD -> adapt-basecam-to-support-cam_anim, origin/adapt-basecam-to-support-cam_anim) added additional print statements to confirm normalization
34f6631 fixed bug related to video naming
f97ed28 resaved notebook
45ec5b7 started local/repo notebook for CAManim
79dbfc3 changed _ffmpeg methods to use subprocess instead
ff9ffb4 fixed bug related to folder not created
763508b trying to fix the bug related to global vs layer norm
56aecc8 fixed image path building
27209c0 added printing of tmp_dir
f92fe9b fixed bug where layer norm was same as global norm


In [4]:

!pip install memory_profiler

In [5]:
%load_ext memory_profiler

In [6]:
!pip install "monai[fire]" "torch"
!python -m monai.bundle download "breast_density_classification" --bundle_dir "../bundles/"

2023-09-30 19:40:29.062441: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-30 19:40:33.245953: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-09-30 19:40:35,955 - INFO - --- input summary of monai.bundle.scripts.download ---
2023-09-30 19:40:35,956 - INFO - > name: 'breast_density_classification'
2023-09-30 19:40:35,956 - INFO - > bundle_dir: '../bundles/'
2023-09-30 19:40:35,956 - INFO - > source: 'github'
2023-09-30 19:40:35,956 - INFO - > remove_prefix: 'monai_'
2023-09-30 19:40:35,956 - INFO - > progress: True
2023-09-30 19:40:35,956 - INFO - ---


breast_density_classification_v0.1.5.zip: 94.5MB [00:01, 63.7MB/s]                
2023-09-30 19:40:38,038 - INFO - Downloaded: ../bundles/bre

## Load image/data

In [7]:
import json
import numpy as np
import cv2
from PIL import Image
import requests
from pytorch_grad_cam.utils.image import preprocess_image
from tqdm import tqdm
import torch

bear image

In [8]:
image_url = "https://th.bing.com/th/id/R.94b33a074b9ceeb27b1c7fba0f66db74?rik=wN27mvigyFlXGg&riu=http%3a%2f%2fimages5.fanpop.com%2fimage%2fphotos%2f31400000%2fBear-Wallpaper-bears-31446777-1600-1200.jpg&ehk=oD0JPpRVTZZ6yizZtGQtnsBGK2pAap2xv3sU3A4bIMc%3d&risl=&pid=ImgRaw&r=0"
img = np.array(Image.open(requests.get(image_url, stream=True).raw))
img = cv2.resize(img, (224, 224))
img = np.float32(img) / 255
input_tensor = preprocess_image(img, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
input_tensor.shape

torch.Size([1, 3, 224, 224])

breast image

In [9]:
# from torchvision.transforms import Compose, Normalize, ToTensor
# def preprocess_image_nonorm(
#     img: np.ndarray, mean=[
#         0.5, 0.5, 0.5], std=[
#             0.5, 0.5, 0.5]) -> torch.Tensor:
#     preprocessing = Compose([
#         ToTensor(),
#         # Normalize(mean=mean, std=std)
#     ])
#     return preprocessing(img.copy()).unsqueeze(0)

In [10]:
image_pth = "/content/bundles/breast_density_classification/sample_data/A/sample_A1.jpg"
img = np.array(Image.open(image_pth))
img = cv2.resize(img, (299, 299))
print(np.max(img))
print(np.min(img))
# img = np.float32(img) / np.max(img)
input_tensor = preprocess_image(img)#, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
print(input_tensor.min())
print(input_tensor.max())
input_tensor.shape

255
0
tensor(-1.)
tensor(1.)


torch.Size([1, 3, 299, 299])

MedNIST

## Define CAM list

In [9]:
# from pytorch_grad_cam import GradCAM, GradCAMPlusPlus, EigenGradCAM, AblationCAM, RandomCAM
from pytorch_grad_cam import EigenCAM, \
                              EigenGradCAM, \
                              FullGrad, \
                              GradCAM, \
                              GradCAMElementWise, \
                              GradCAMPlusPlus, \
                              HiResCAM, \
                              LayerCAM, \
                              RandomCAM, \
                              ScoreCAM, \
                              XGradCAM, \
                              AblationCAM


cams_list = [#'AblationCAM', # ran out of RAM
            'GradCAM',
            'HiResCAM',
            'GradCAMElementWise',
            'GradCAMPlusPlus',
            'EigenCAM', # done
            #'EigenGradCAM', # Skipped all layers for AlexNet; figure out
            'RandomCAM',
            'LayerCAM',
             'XGradCAM',
             #'FullGrad', # Skipped all layers for DenseNet161; figure out
             #'ScoreCAM', # ran out of RAM
             ]

## Define google drive location

We connect google drive to save the generated CAManim picutres and videos. The root of the save directory is defined by the variable `save_root`

In [10]:
from google.colab import drive
# drive.mount("/content/gdrive", force_remount=True)
drive.mount("/content/gdrive")
drive_path = '/content/gdrive/MyDrive'

from pathlib import Path

save_root = Path(drive_path) / "starCAManim"

Mounted at /content/gdrive


## Define models dictionary

breast density model

In [ ]:
# load monai breast density model
from monai.networks.nets import TorchVisionFCModel
model_path = "/content/bundles/breast_density_classification/models/model.pt"
breast_model = TorchVisionFCModel(model_name="inception_v3", num_classes=4, pool=None) # ref for params -> https://github.com/Project-MONAI/model-zoo/blob/dev/models/breast_density_classification/configs/inference.json
breast_model.load_state_dict(torch.load(model_path))
breast_model.eval()

torchvision pretrained model

In [13]:
from torchvision import models # Only needed for example model, not overall anim code
# model = models.densenet161(pretrained=True)# List available models
all_models = models.list_models()
classification_models = models.list_models(module=models)
classification_models

['alexnet',
 'convnext_base',
 'convnext_large',
 'convnext_small',
 'convnext_tiny',
 'densenet121',
 'densenet161',
 'densenet169',
 'densenet201',
 'efficientnet_b0',
 'efficientnet_b1',
 'efficientnet_b2',
 'efficientnet_b3',
 'efficientnet_b4',
 'efficientnet_b5',
 'efficientnet_b6',
 'efficientnet_b7',
 'efficientnet_v2_l',
 'efficientnet_v2_m',
 'efficientnet_v2_s',
 'googlenet',
 'inception_v3',
 'maxvit_t',
 'mnasnet0_5',
 'mnasnet0_75',
 'mnasnet1_0',
 'mnasnet1_3',
 'mobilenet_v2',
 'mobilenet_v3_large',
 'mobilenet_v3_small',
 'regnet_x_16gf',
 'regnet_x_1_6gf',
 'regnet_x_32gf',
 'regnet_x_3_2gf',
 'regnet_x_400mf',
 'regnet_x_800mf',
 'regnet_x_8gf',
 'regnet_y_128gf',
 'regnet_y_16gf',
 'regnet_y_1_6gf',
 'regnet_y_32gf',
 'regnet_y_3_2gf',
 'regnet_y_400mf',
 'regnet_y_800mf',
 'regnet_y_8gf',
 'resnet101',
 'resnet152',
 'resnet18',
 'resnet34',
 'resnet50',
 'resnext101_32x8d',
 'resnext101_64x4d',
 'resnext50_32x4d',
 'shufflenet_v2_x0_5',
 'shufflenet_v2_x1_0',
 'sh

In [14]:
models_dict = {
                # "wide_resnet101_2": models.wide_resnet101_2(pretrained=True),
                "densenet121": models.densenet161(pretrained=True),
                # "breast_density_classification": breast_model
              }

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet161_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet161_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/densenet161-8d451a50.pth" to /root/.cache/torch/hub/checkpoints/densenet161-8d451a50.pth
100%|██████████| 110M/110M [00:00<00:00, 179MB/s]


### visualize model parameters


In [15]:
model = models_dict["densenet121"]
layers = [layer for layer, what_is_this in model.named_modules()]
print(len(layers))
parameters = [p for p in model.parameters()]# if p.requires_grad]
len(parameters)

573


484

In [16]:
running_sum = 0

total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

for layer_name, layer_module in model.named_modules():

  # print("-"*6 + layer_name + "-"*6)
  # print(type(layer_name))
  # print(type(layer_module))
  # layer_parameters = [parameters = [p for p in model.parameters()]# if p.requires_grad]]
  n_layer_params = sum(p.numel() for p in layer_module.parameters() if p.requires_grad)
  running_sum += n_layer_params

print(running_sum)
print(total_params)

134135120
28681000


## get target_layer

In [17]:

def get_target_layers(model):
  """
  target_layers doesn't matter for cam_anim (since we loop through all the layers),
  but it is needed for initializing the GradCAM object.
  This method returns the last layer (that is viable as a target i.e. iterable) of the model.

  """


  target_layers = []
  for _, layer_module in model.named_modules():
    # print(_)
    if _ == "features.Mixed_7c.branch_pool.conv":
      target_layers.append(layer_module)
    # https://stackoverflow.com/questions/1952464/in-python-how-do-i-determine-if-an-object-is-iterable
    try:
      some_object_iterator = iter(layer_module)
      if type(layer_module)!=str:
        target_layers.append(layer_module)

    except TypeError as te:
      pass
      # print(some_object, 'is not iterable')
  # target_layers = target_layers[-2]
  print(target_layers)
  return target_layers

## Double Loop - n_models X n_cams

In [18]:
n_models = len(models_dict.keys())
n_cams = len(cams_list)

print(f"{n_models} X {n_cams} = {n_models*n_cams} iterations")

1 X 8 = 8 iterations


In [19]:
from pytorch_grad_cam.base_cam import BaseCAM
import pandas as pd

import gc
gc.enable()

In [20]:
print(gc.collect())
print(pd.DataFrame.from_records(gc.get_stats()))

0
   collections  collected  uncollectable
0          834       2694              0
1           75        838              0
2            7         68              0


In [21]:
!ffmpeg -version

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enable-l

In [ ]:
# %%mprun -f BaseCAM.cam_anim

metrics_records = []
for model_name, model in models_dict.items():

  print("="*33 + model_name + "="*33)

  outdir = Path(drive_path) / "starCAManim" / model_name
  outdir.mkdir(exist_ok=True, parents= True)

  target_layers = get_target_layers(model)
  # target_layers = ["features.Mixed_7c.branch_pool.conv"]

  for cam_name in cams_list:
    print("-"*33 + cam_name + "-"*33)

    frames_dir = outdir / cam_name
    frames_dir.mkdir(exist_ok=True, parents= True)
    metrics = {"model_name": model_name, "CAM Name": cam_name, "error": None}
    try:
      cam = globals()[cam_name](model=model, target_layers=target_layers, use_cuda=False)
      metrics_update = cam.cam_anim(img,
                                    input_tensor,
                                    frame_rate=24,
                                    norm_type='both',
                                    keep_frames=True,
                                    tmp_dir=str(frames_dir),
                                    output_fname=str(outdir / f"{cam_name}_anim.mp4"))

      metrics.update(metrics_update)
      del cam
      gc.collect()
      print(pd.DataFrame.from_records(gc.get_stats()))
      print(f"{cam_name} Finished")


    except Exception as ex:
      metrics.update({"error": str(ex)})
      print(ex)
      print("Continuing to the next CAM")
      raise ex

    metrics_records.append(metrics)

    with open(str(Path(drive_path) / "starCAManim" / f"metrics_logs_{model_name}.json"), "w") as outfile:
      json.dump(metrics_records, outfile, indent=4)

=================================densenet121=================================
[Sequential(
  (conv0): Conv2d(3, 96, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (norm0): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu0): ReLU(inplace=True)
  (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (denseblock1): _DenseBlock(
    (denselayer1): _DenseLayer(
      (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu1): ReLU(inplace=True)
      (conv1): Conv2d(96, 192, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (norm2): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu2): ReLU(inplace=True)
      (conv2): Conv2d(192, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    )
    (denselayer2): _DenseLayer(
      (norm1): BatchNorm2d(144, eps=1e-05, momentum=0.1, affine=True, track_running

Model Layer Loop - features:   0%|          | 0/573 [00:00<?, ?it/s]

skipping 
'DenseNet' object has no attribute ''


Model Layer Loop - features.denseblock4.denselayer15.norm2:  88%|████████▊ | 505/573 [35:18<02:16,  2.00s/it]

## Generate Videos Only

Run this section if you have all the frames and want to regenerate the video

In [ ]:
import subprocess
def _ffmpeg_standard_quality(tmp_path, output_fname, frame_rate=5):
  """ _ffmpeg_standard_quality
      Generates and saves-to-file the animated .MP4 video in standard quality.
      ---
      Input: tmp_path <str>, the path to the images
             output_fname <str>, the path and filename for the saved file
             frame_rate=5 <int>, the number of frames per second
      Output: None
  """
  print('Generating video with pngs from:\n', tmp_path)
  try:
    # Define the command
    command = ['ffmpeg', '-framerate', str(frame_rate), '-pattern_type', 'glob', '-i', tmp_path + '/*.png', '-c:v', 'libx264', '-pix_fmt', 'yuv420p', output_fname]

    # Run the command and capture all output channels
    result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True)

    # Print the output
    print(result.stdout)
    print(result.stderr)
  except Exception as exc:
    print('ERROR: ffmpeg video generation failed; video corrupt.')
    print(exc)
  return None

In [ ]:
metrics_records = []
for model_name, model in models_dict.items():

  print("="*33 + model_name + "="*33)

  outdir = Path(drive_path) / "starCAManim" / model_name
  outdir.mkdir(exist_ok=True, parents= True)

  target_layers = get_target_layers(model)
  # target_layers = ["features.Mixed_7c.branch_pool.conv"]

  for cam_name in cams_list:
    print("-"*33 + cam_name + "-"*33)

    frames_dir = outdir / cam_name
    frames_dir.mkdir(exist_ok=True, parents= True)
    metrics = {"model_name": model_name, "CAM Name": cam_name, "error": None}
    for norm_type in ["global", "layer"]:
      tmp_path = str(outdir / cam_name / norm_type)
      output_fname = str(outdir / f"{cam_name}_{norm_type}_anim.mp4")
      # !ffmpeg -framerate 24 -pattern_type glob -i '<tmp_path>/*.png' -c:v libx264 -pix_fmt yuv420p $output_fname
      _ffmpeg_standard_quality(tmp_path, output_fname, frame_rate=24)

## Load logged metrics

In [ ]:

# Opening JSON file
model_name = "breast_density_classification"
with open(str(Path(drive_path) / "starCAManim" / f"metrics_logs_{model_name}.json"), 'r') as openfile:
    # Reading from json file
    metrics_records = json.load(openfile)

In [ ]:
import pandas as pd

metrics_df = pd.DataFrame.from_records(metrics_records)
display(metrics_df)

In [ ]:
def unpack_layer_records(row):

  layer_df = pd.DataFrame.from_records(row["layers_records"])

  for col in row.index:
    if col not in ["layers_records", "error"]:
      layer_df[col] = row[col]

    layer_df["cam_anim_error"] = row["error"]

  return layer_df


In [ ]:
import pandas as pd
df = pd.DataFrame({
  'date': ['2022-09-14', '2022-09-15', '2022-09-16'],
  'letter': ['A', 'B', 'C'],
  'dict' : [{ 'fruit': 'apple', 'weather': 'aces'},
            { 'fruit': 'banana', 'weather': 'bad'},
            { 'fruit': 'cantaloupe', 'weather': 'cloudy'}],
})

pd.concat([df.drop(['dict'], axis=1), df['dict'].apply(pd.Series)], axis=1)

In [ ]:
layer_metrics_df = None

for _, row in metrics_df.iterrows():
  layer_df = unpack_layer_records(row)

  if layer_metrics_df is None:
    layer_metrics_df = layer_df
  else:
    layer_metrics_df = pd.concat([layer_metrics_df, layer_df])


display(layer_metrics_df)
# layer_metrics_df = layer_metrics_df.groupby("CAM Name")
layer_metrics_df["layer_time"].describe()

## Plot

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import pandas as pd

df = layer_metrics_df.copy(deep=True)
# df = df[df["CAM Name"]=="HiResCAM"]

plt.figure(figsize=(9,9))
sns.scatterplot(x="layer_num_parameters", y="layer_time",
                # s=100,
                # figsize=(64,64),
                hue='CAM Name',
                #style='zone_number',
                data=df)

plt.xscale('log')
plt.yscale('log')
plt.show()